# Scraping raw data from the main units of Brønnøysund
This is with the aim of using role data e.g daglig leder / direktør / styremedlem etc  
The scraping is done using main and parent unit orgnumber from the unique_brønnøysund_company_data.obj file.  

No processing of the data is done in this file, only gattering the raw data and storing it.

In [50]:
# Library imports
import pickle
import requests
import os.path as path

In [5]:
# Test of the scraper response.
url = "https://w2.brreg.no/enhet/sok/detalj.jsp?orgnr=932812835"
reponse = requests.get(url)

if reponse.ok:
    print(reponse.text)

In [6]:
# Importing the orgnumber data:
# Gets the parent directory, enabeling direct targeting of the desiref file.

parent = path.abspath(path.join("unique_brønnøysund_company_data.obj" ,"../../.."))
with open(f'{parent}/Data/Raw/unique_brønnøysund_company_data.obj', 'rb') as file:
	unique_org_num = pickle.load(file)

In [7]:
for key, value in unique_org_num.items():
    print(f"{key} its lenght is: {len(value)}")

sub_unit its lenght is: 1152
main_unit its lenght is: 11
parent_unit its lenght is: 992


In [25]:
def scraping_brønnøysund(list_of_orgnumbers):
    """Gets a list of html strings based on a input list of orgnumbers

    Returns:
        list: list of html stings, needs a html phaser
    """

    data_response_list = []    
    for org_number in list_of_orgnumbers:
        url = "https://w2.brreg.no/enhet/sok/detalj.jsp?orgnr="
        reponse = requests.get(url + str(org_number))
        if reponse.ok:
            data_response_list.append(reponse.text)
        else:
            print(f"Could not get data for {org_number}, status code: {reponse.status_code}")
    return data_response_list

In [16]:
# Renaming the orgnumber variabels
main_units_org_nums = unique_org_num['main_unit']
parent_units_org_nums = unique_org_num['parent_unit']
print(len(main_units_org_nums))
print(len(parent_units_org_nums))

11
992


In [31]:
# Scraping the data for the 11 main units
scraped_data_main = scraping_brønnøysund(main_units_org_nums)
print(len(scraped_data_main))

11


In [32]:
# Scraping the data for the 992 parent units
scraped_data_parent = scraping_brønnøysund(parent_units_org_nums)
print(len(parent_units_org_nums))

992


#### Storing the data.

In [47]:
# Fulldata
scraped_data_parent.extend(scraped_data_main)

In [51]:
# Store the raw data as a pickle file.
# Gets the parent directory location enabeling storing the 
# data directly to the desired folder.
parent = path.abspath(path.join("brønnøysund_api.ipynb", "../../.."))
path = "/Data/Raw/"
with open(f'{parent + path}scraped_data_from_brønnøysund.obj', 'wb') as file:
	pickle.dump(scraped_data_parent, file)